In [3]:
env = 'colab' if 'google.colab' in str(get_ipython()) else 'local'

if env == 'colab':
    print('Running on CoLab')

    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    import sys
    FOLDERNAME = 'Learning/ATS-Yandex/ROCKET/'
    sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
    
    %cd drive/My\ Drive/$FOLDERNAME/
    input_path = r"/content/drive/My Drive/Learning/ATS-Yandex/ROCKET/data"
else:
    print('Running locally')
    input_path = 'data'

Running on CoLab
Mounted at /content/drive
/content/drive/My Drive/Learning/ATS-Yandex/ROCKET


In [4]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader 
import torch.nn as nn
from torch import optim, Tensor
from sklearn.linear_model import RidgeClassifierCV
import time, datetime

In [5]:
class UCRDataset(Dataset):
    """Dataset which samples the data from hourly electricity data."""

    def __init__(self, data, normalize=True, unsqueeze=True):
        self.raw_data = data
        Y, X = data[:, 0].astype(np.int32), data[:, 1:]
        if normalize:
            X = (X - X.mean(axis=1, keepdims=True)) / (X.std(axis=1, keepdims=True) + 1e-8)

        X = torch.Tensor(X)

        if unsqueeze and len(X.shape) < 3:
            X = X.unsqueeze(1)
        self.X = X
        self.Y = Y

    def __len__(self):
        return self.raw_data.shape[0]

    def __getitem__(self, idx):
        sample = self.X[idx, :]
        label = int(self.Y[idx] == 1)
        return sample, label

    def __seqlen__(self):
        return self.raw_data.shape[1]

In [6]:
class ROCKET(nn.Module):

    def __init__(self, in_channels, seq_len, num_kernels=10000, kernel_sizes=[7, 9, 11], device=None):
        super().__init__()
        kernels = nn.ModuleList()

        for i in range(num_kernels):
            kernel_size = np.random.choice(kernel_sizes)
            dilation = 2 ** np.random.uniform(0, np.log2((seq_len - 1) // (kernel_size - 1)))
            padding = int((kernel_size - 1) * dilation // 2) if np.random.randint(2) == 1 else 0

            weight = torch.randn(1, in_channels, kernel_size)
            weight -= weight.mean()
            bias = 2 * torch.rand(1) - 1

            kernel = nn.Conv1d(in_channels, 1, kernel_size, padding=2 * padding, dilation=int(dilation), bias=True)
            kernel.weight = torch.nn.Parameter(weight, requires_grad=False)
            kernel.bias = torch.nn.Parameter(bias, requires_grad=False)

            kernels.append(kernel)

        self.kernels = kernels
        self.num_kernels = num_kernels
        self.kss = kernel_sizes

    def forward(self, x):
        features = []
        for i in range(self.num_kernels):
            output = self.kernels[i](x).cpu()
            kernel_max, _ = output.max(dim=-1)
            kernel_ppv = torch.gt(output, 0).sum(dim=-1).float() / output.shape[-1]
            features.append(kernel_max)
            features.append(kernel_ppv)
        output = torch.cat(features, dim=1)
        return output

In [8]:
def load_data(dataset_name):
    train_data = np.loadtxt(f"{input_path}/{dataset_name}/{dataset_name}_TRAIN.txt")
    test_data = np.loadtxt(f"{input_path}/{dataset_name}/{dataset_name}_TEST.txt")

    train_ds = UCRDataset(train_data)
    test_ds = UCRDataset(test_data)
    
    return train_ds, test_ds

In [9]:
dataset_names = ['FordA']
num_runs = 1
in_channels = 1
num_kernels = 1000

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

results = pd.DataFrame(index=dataset_names,
                       columns=["accuracy_mean",
                                "accuracy_standard_deviation",
                                "time_training_seconds",
                                "time_test_seconds"],
                       data=0)
results.index.name = "dataset"

In [22]:
for dataset_name in dataset_names:

    print(f"{dataset_name}".center(80, "-"))

    # -- read data -------------------------------------------------------------
    train_ds, test_ds = load_data(dataset_name)
    seq_len = train_ds.__seqlen__()

    _results = np.zeros(num_runs)
    _timings = np.zeros([4, num_runs])

    X_train, Y_train = train_ds.X, train_ds.Y
    X_test, Y_test = test_ds.X, test_ds.Y

    X_train = X_train.to(device)
    X_test = X_test.to(device)

    # -- run -------------------------------------------------------------------

    print(f"Performing runs".ljust(80 - 5, "."), end="", flush=True)

    for i in range(num_runs):

        conv_model = ROCKET(in_channels, seq_len, num_kernels=num_kernels).to(device)

        # -- transform training ------------------------------------------------
        time_a = time.perf_counter()
        X_train_transform = conv_model(X_train).cpu()
        time_b = time.perf_counter()
        _timings[0, i] = time_b - time_a
        print("\nconv1d transform train " + str(time_b - time_a))

        # -- transform test ----------------------------------------------------
        time_a = time.perf_counter()
        X_test_transform = conv_model(X_test).cpu()
        time_b = time.perf_counter()
        _timings[1, i] = time_b - time_a
        print("conv1d transform test " + str(time_b - time_a))

        # -- training ----------------------------------------------------------
        time_a = time.perf_counter()
        classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
        classifier.fit(X_train_transform, Y_train)
        time_b = time.perf_counter()
        _timings[2, i] = time_b - time_a
        print("training Rifge classifier " + str(time_b - time_a))

        # -- test --------------------------------------------------------------
        time_a = time.perf_counter()
        _results[i] = classifier.score(X_test_transform, Y_test)
        time_b = time.perf_counter()
        _timings[3, i] = time_b - time_a
        print("Test evaluation " + str(time_b - time_a))
    print("Done.")

    results.loc[dataset_name, "accuracy_mean"] = _results.mean()
    results.loc[dataset_name, "accuracy_standard_deviation"] = _results.std()
    results.loc[dataset_name, "time_training_seconds"] = _timings.mean(1)[[0, 2]].sum()
    results.loc[dataset_name, "time_test_seconds"] = _timings.mean(1)[[1, 3]].sum()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(results)


-------------------------------------FordA--------------------------------------
Performing runs............................................................
conv1d transform train 10.465163385000324
conv1d transform test 3.7843717979999383
training Rifge classifier 9.12444040499986
Test evaluation 0.008656824999889068
Done.
         accuracy_mean  accuracy_standard_deviation  time_training_seconds  \
dataset                                                                      
FordA         0.937121                          0.0              19.589604   

         time_test_seconds  
dataset                     
FordA             3.793029  


In [21]:
for dataset_name in dataset_names:

    print(f"{dataset_name}".center(80, "-"))

    # -- read data -------------------------------------------------------------
    train_ds, test_ds = load_data(dataset_name)
    seq_len = train_ds.__seqlen__()

    _results = np.zeros(num_runs)
    _timings = np.zeros([4, num_runs])
        
    batch_size = 64

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

    def batch_transform(data_loader):
        X_transform = []
        Y = []
        for _, (X, y) in enumerate(data_loader):
            X_transform.append(conv_model(X.to(device)))
            Y.append(y)
        X_transform = torch.cat(X_transform, dim=0)
        Y = torch.cat(Y, dim=0)
        return X_transform, Y

    for i in range(num_runs):
        conv_model = ROCKET(in_channels, seq_len, num_kernels=num_kernels).to(device)

        # -- transform training ------------------------------------------------
        time_a = time.perf_counter()
        X_train_transform, Y_train = batch_transform(train_loader)
        time_b = time.perf_counter()
        _timings[0, i] = time_b - time_a

        # -- transform test ----------------------------------------------------
        time_a = time.perf_counter()
        X_test_transform, Y_test = batch_transform(test_loader)
        time_b = time.perf_counter()
        _timings[1, i] = time_b - time_a

        # -- Normalizing Features ----------------------------------------------
        f_mean = X_train_transform.mean(axis=0, keepdims=True)
        f_std = X_train_transform.std(axis=0, keepdims=True) + 1e-6

        X_train_transform = (X_train_transform - f_mean) / f_std
        X_test_transform = (X_test_transform - f_mean) / f_std

        # -- Dimensionality Reduction-------------------------------------------
        from sklearn.decomposition import PCA
        # num_features = min(2 * num_kernels, train_ds.__len__()) - 1
        pca = PCA(n_components=0.99)
        X_train_transform = torch.FloatTensor(pca.fit_transform(X_train_transform))
        X_test_transform = torch.FloatTensor(pca.transform(X_test_transform))
        num_features = pca.n_components_
        print("num components=" + str(num_features))

        # -- training ----------------------------------------------------------

        time_a = time.perf_counter()

        train_features_ds = TensorDataset(X_train_transform, Y_train)
        train_ft_loader = DataLoader(train_features_ds, batch_size=batch_size, shuffle=False)

        linear_model = nn.Linear(num_features, len(torch.unique(Y_train))).to(device)
        loss_function = nn.CrossEntropyLoss()
        optimizer = optim.Adam(linear_model.parameters(), lr=5e-5, weight_decay=0.02)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, min_lr=1e-8)
        
        epochs = 250
        print_every = 50
        
        for epoch in range(epochs):
            # Train Loop
            linear_model.train()
            for t, (X, y) in enumerate(train_ft_loader):
                X, y = X.to(device), y.to(device)
                y_pred = linear_model(X)
                loss = loss_function(y_pred, y)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            # Eval Loop
            linear_model.eval()
            y_pred = linear_model(X_test_transform.to(device)).to('cpu')
            loss = loss_function(y_pred, Y_test)
            acc = (y_pred.max(1)[1].numpy() == Y_test.numpy()).mean()
            if epoch % print_every == 0:
                print(str(datetime.datetime.now()),
                      'Epoch=%d' % epoch,
                      ' Valid. avg loss=%.4f' % loss.item(), 
                      ' Valid. avg acc=%.4f' % acc, 
                      'last_lr=', optimizer.param_groups[0]['lr'])
            if scheduler:
                scheduler.step(loss)
        _results[i] = (y_pred.max(1)[1].numpy() == Y_test.numpy()).mean()
        time_b = time.perf_counter()
        _timings[2, i] = time_b - time_a

    print("Done.")
    
    results.loc[dataset_name, "accuracy_mean"] = _results.mean()
    results.loc[dataset_name, "accuracy_standard_deviation"] = _results.std()
    results.loc[dataset_name, "time_training_seconds"] = _timings.mean(1)[[0, 2]].sum()
    results.loc[dataset_name, "time_test_seconds"] = _timings.mean(1)[[1, 3]].sum()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(results)


-------------------------------------FordA--------------------------------------
num components=1199
2021-03-23 00:32:08.571966 Epoch=0  Valid. avg loss=0.8480  Valid. avg acc=0.4273 last_lr= 5e-05
2021-03-23 00:32:12.789240 Epoch=50  Valid. avg loss=0.2112  Valid. avg acc=0.9402 last_lr= 5e-05
2021-03-23 00:32:16.901908 Epoch=100  Valid. avg loss=0.1599  Valid. avg acc=0.9379 last_lr= 5e-05
2021-03-23 00:32:21.081032 Epoch=150  Valid. avg loss=0.1534  Valid. avg acc=0.9379 last_lr= 5e-05
2021-03-23 00:32:25.283047 Epoch=200  Valid. avg loss=0.1537  Valid. avg acc=0.9379 last_lr= 3.125e-06
Done.
         accuracy_mean  accuracy_standard_deviation  time_training_seconds  \
dataset                                                                      
FordA         0.937879                          0.0              40.089094   

         time_test_seconds  
dataset                     
FordA             7.015341  
